**TITRE**

In [5]:
import math

class AbstractRule:
    
    def _set_grammar(self,gram):
        self._grammar = gram
        
    def valuation(self):
        raise NotImplementedError
        
    def count(self):
        raise NotImplementedError
        
    def listR(self,i):
        raise NotImplementedError
        
    def unrank(self,n,i):
        raise NotImplementedError

        
#Exception triggered when a rule uses a literal not in the grammar
class UnknownLiteralError(Exception):
    
    def __init__(self,literal):
        self.literal = literal
        
#Exception triggered when an unrank goes wrong
class ValueError(Exception):
    pass

        
class ConstructorRule(AbstractRule):
    
    def __init__(self, *args):
        self._parameters = args
        #print (self._parameters[0])
        #print (self._parameters[1])
        self._valuation = math.inf

    def valuation(self):
        return self._valuation
    
    #The Grammar is supposed non ambiguous
    #check if all literals used in the rule are parts of the grammar
    def _verif_rule(self):
        try:
            self._grammar[self._parameters[0]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[0])
        try:
            self._grammar[self._parameters[1]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[1])

    #Return true if there was no update
    def _update_valuation(self):
            #print("Valuation : Constructor")
            #print("old :"+str(self._valuation))
            self._old_val = self._valuation
            self._valuation = self._calc_valuation()
            #print("new :"+str(self._valuation))
            #print(self._old_val == self._valuation)
            return (self._old_val == self._valuation)

        
class UnionRule(ConstructorRule):
    
    def __init__(self,fst,snd):
        #print ("union")
        super().__init__(fst,snd)
        
    def _calc_valuation(self):
        #print ("union")
        return min(self._grammar[self._parameters[0]].valuation(),
                   self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print (self._grammar[self._parameters[0]].count(i))
        #print (self._grammar[self._parameters[1]].count(i))
        return self._grammar[self._parameters[0]].count(i) + self._grammar[self._parameters[1]].count(i)
    
    def listR(self, i):
        return self._grammar[self._parameters[0]].listR(i) + self._grammar[self._parameters[1]].listR(i)
    
    def unrank(self,n,i):
        r0 = self._grammar[self._parameters[0]]
        r1 = self._grammar[self._parameters[1]]
        c0 = r0.count(n)
        if i < c0:
            return r0.unrank(n,i)
        else:
            return r1.unrank(n,i-c0)
    
        
class ProductRule(ConstructorRule):
    
    def __init__(self,fst,snd,cons):
        #print("product")
        super().__init__(fst,snd)
        self._constructor = cons
        
    def _calc_valuation(self):
        #print("product")
        return (self._grammar[self._parameters[0]].valuation() +
               self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print("count : product")
        res = 0
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print("i :" + str(i))
        #print("valN1 :" + str(valN1))
        #print("valN2 :" + str(valN2))
        # WARNING
        # This might break some grammars (weird grammars with 2 different epsilon rules)
        # but avoid some infinite recursion (e.g. Dyck grammar)
        
        #if i == 0 and valN1 == 0 and valN2 == 0:
         #   return 1
        for k in range (valN1, i-valN2+1):
            #print("k : "+str(k))
            #print("l : "+str(i-k))
            res += self._grammar[self._parameters[0]].count(k) * self._grammar[self._parameters[1]].count(i-k)
            #print("res : "+str(res))
        return res
    
    def listR(self, i):
        #print("listR : product")
        res = []
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print("i :" + str(i))
        #print("valN1 :" + str(valN1))
        #print("valN2 :" + str(valN2))
        # WARNING
        # This might break some grammars (weird grammars with 2 different epsilon rules)
        # but avoid some infinite recursion (e.g. Dyck grammar)
        
        #if i == 0 and valN1 == 0 and valN2 == 0:
         #   return 1
        for k in range (valN1, i-valN2+1):
            #print("k : "+str(k))
            #print("l : "+str(i-k))
            list0 = self._grammar[self._parameters[0]].listR(k)
            list1 = self._grammar[self._parameters[1]].listR(i-k)
            res += [self._constructor(x,y) for x in list0 for y in list1]
            
            #print("res : "+str(res))
        return res

    def unrank(self,n,i):
        if(i>=self.count(n)):
            raise ValueError
        r0 = self._grammar[self._parameters[0]]
        r1 = self._grammar[self._parameters[1]]
        valN0 = r0.valuation()
        valN1 = r1.valuation()
        k = valN0
        while k<n-valN1+1:
            c0 = r0.count(k)
            c1 = r1.count(n-k)
            if(i<c0*c1)
    
    
class ConstantRule(AbstractRule):
    
    def __init__(self,obj):
        self._object = obj
        
    #We chose to put _update_valuation here too
    #to avoid testing the presence of the function
    #every time we want to call it on a rule
    def _update_valuation(self):
        return True

    
class EpsilonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
    
    def valuation(self):
        return 0
    
    def count(self, i):
        if i != 0:
            return 0
        else :
            return 1
        
    def listR(self,i):
        if i == 0:
            return [self._object]
        else:
            return []
        
    def unrank(self,n,i):
        if i == 0 :
            return self._object
        else:
            raise ValueError
    
class SingletonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
        
    def valuation(self):
        return 1
    
    def count(self, i):
        if i != 1:
            return 0
        else :
            return 1
        
    def listR(self,i):
        if i == 1:
            return [self._object]
        else:
            return []
        
    def unrank(self,n,i):
        if i==0:
            return self._object
        else:
            raise ValueError

#Triggered when a non terminal does not generate anything
class CircularGrammarError(Exception):
    
    def __init__(self,grammar,name):
        self.name = name
        self.grammar = grammar
        
    
def init_grammar(gram):
    #set the grammar for all rules
    for rule in gram.values() :
        rule._set_grammar(gram)
        #verifies every non-constant rule (check if all literal used in the rules are in the grammar)
        if isinstance(rule, ConstructorRule):
            rule._verif_rule()

    #As long as there's a change we update again
    #(we chose to make _update_valuation available
    #to any rule)
    while not all(rule._update_valuation() for rule in gram.values()):
        pass
    for name,rule in gram.items() :
        if rule.valuation() == math.inf :
            raise CircularGrammarError(gram,name)

In [174]:
#Timeout for the tests
import signal

class TimeoutError(Exception):
    pass

def timeout(signum,frame):
    raise TimeoutError
    
signal.signal(signal.SIGALRM, timeout)

#TODOCorrect
#Testing all the list with count (in a limited time stamp)
"""
@param grammar : the grammar which is tested
@param base : the litteral that generates elements of the grammar
"""
def test_Count_List_Until_Timeout (grammar, base):
    
    i = 0
    try:
        while True:
            signal.alarm(1)
            l = grammar[base].listR(i)
            c = grammar[base].count(i)
            signal.alarm(0)
            assert (c == len(l))
            #Check if there is no multiple appearance of the same value
            assert (len(l) == len(set(l)))
            #print(i)
            i+=1
    except TimeoutError :
        print("the greatest test was for i = " + str(i-1))


#Common imports to all the tests
import operator

In [175]:
#Grammar definition for binary trees
treeGram = {"Tree" : UnionRule("Node","Leaf"),
            "Node" : ProductRule("Tree","Tree",
                                lambda a, b : "("+a+","+b+")"),
            "Leaf" : SingletonRule("Leaf")}
init_grammar(treeGram)

In [176]:
#tests for the binary trees grammar

#valuation

assert (treeGram["Tree"].valuation() == 1)
assert (treeGram["Node"].valuation() == 2)
assert (treeGram["Leaf"].valuation() == 1)

#count

assert (treeGram["Leaf"].count(0) == 0)
assert (treeGram["Leaf"].count(1) == 1)
assert (treeGram["Leaf"].count(2) == 0)
assert (treeGram["Tree"].count(0) == 0)
assert (treeGram["Tree"].count(1) == 1)
assert (treeGram["Tree"].count(2) == 1)
assert (treeGram["Tree"].count(3) == 2)
assert (treeGram["Tree"].count(4) == 5)
assert (treeGram["Tree"].count(5) == 14)
assert (treeGram["Tree"].count(6) == 42)
assert (treeGram["Tree"].count(7) == 132)
assert (treeGram["Tree"].count(8) == 429)
assert (treeGram["Tree"].count(9) == 1430)


#listR
assert (treeGram["Tree"].listR(0) == [])
assert (treeGram["Tree"].listR(1) == ["Leaf"])
assert (set(treeGram["Tree"].listR(2)) == set(["(Leaf,Leaf)"]))
assert (set(treeGram["Tree"].listR(3)) == set(["(Leaf,(Leaf,Leaf))",
                                      "((Leaf,Leaf),Leaf)"]))
assert (set(treeGram["Tree"].listR(4)) == set(["((Leaf,(Leaf,Leaf)),Leaf)",
                                      "(Leaf,((Leaf,Leaf),Leaf))",
                                      "((Leaf,Leaf),(Leaf,Leaf))",
                                      "(Leaf,(Leaf,(Leaf,Leaf)))",
                                      "(((Leaf,Leaf),Leaf),Leaf)"
                                  ]))
test_Count_List_Until_Timeout(treeGram, "Tree")

the greatest test was for i = 12


In [177]:
#Grammar definition for Fibonacci words
fiboGram = {"Fib" : UnionRule("Vide", "Cas1"),
            "Cas1" : UnionRule("CasAu", "Cas2"),
            "Cas2" : UnionRule("AtomB", "CasBAu"),
            "Vide" : EpsilonRule(""),
            "CasAu" : ProductRule("AtomA", "Fib",operator.add),
            "AtomA" : SingletonRule("A"),
            "AtomB" : SingletonRule("B"),
            "CasBAu" : ProductRule("AtomB", "CasAu",operator.add)}
init_grammar(fiboGram)

In [196]:
#tests for the fibonacci words grammar

#valuation

assert (fiboGram["Fib"].valuation() == 0)
assert (fiboGram["Cas1"].valuation() == 1)
assert (fiboGram["Cas2"].valuation() == 1)
assert (fiboGram["Vide"].valuation() == 0)
assert (fiboGram["CasAu"].valuation() == 1)
assert (fiboGram["AtomA"].valuation() == 1)
assert (fiboGram["AtomB"].valuation() == 1)
assert (fiboGram["CasBAu"].valuation() == 2)

#count

assert (fiboGram["Fib"].count(0) == 1)
assert (fiboGram["Fib"].count(1) == 2)
assert (fiboGram["Fib"].count(2) == 3)
assert (fiboGram["Fib"].count(3) == 5)
assert (fiboGram["Fib"].count(4) == 8)
assert (fiboGram["Fib"].count(5) == 13)
assert (fiboGram["Fib"].count(6) == 21)
assert (fiboGram["Fib"].count(7) == 34)
assert (fiboGram["Fib"].count(8) == 55)
assert (fiboGram["Fib"].count(9) == 89)

#listR

assert (fiboGram["Fib"].listR(0) == [""])
assert (set(fiboGram["Fib"].listR(1)) == set(["A","B"]))
assert (set(fiboGram["Fib"].listR(2)) == set(["AA","AB","BA"]))
assert (set(fiboGram["Fib"].listR(3)) == set(["AAA","AAB","ABA","BAA","BAB"]))
assert (set(fiboGram["Fib"].listR(4)) == set(["AAAA","AAAB","AABA","ABAA","ABAB","BAAA",
                                              "BAAB","BABA"]))
test_Count_List_Until_Timeout(fiboGram, "Fib")

the greatest test was for i = 12


In [179]:
#Grammar definition for all the words with A and B letters
abGram = {"Words" : UnionRule("Vide","Cas1"),
          "Cas1" : UnionRule("CasAu","CasBu"),
          "AtomA" : SingletonRule("A"),
          "AtomB" : SingletonRule("B"),
          "CasAu" : ProductRule("AtomA", "Words",operator.add),
          "CasBu" : ProductRule("AtomB", "Words",operator.add),
          "Vide" : EpsilonRule(""),
          }
init_grammar(abGram)

In [180]:
# tests for the (a,b) words grammar

#valuation

assert (abGram["Words"].valuation() == 0)
assert (abGram["Cas1"].valuation() == 1)
assert (abGram["AtomA"].valuation() == 1)
assert (abGram["AtomB"].valuation() == 1)
assert (abGram["CasAu"].valuation() == 1)
assert (abGram["CasBu"].valuation() == 1)
assert (abGram["Vide"].valuation() == 0)

#count

assert (abGram["Words"].count(0) == 1)
assert (abGram["Words"].count(1) == 2)
assert (abGram["Words"].count(2) == 4)
assert (abGram["Words"].count(3) == 8)
assert (abGram["Words"].count(4) == 16)
assert (abGram["Words"].count(5) == 32)
assert (abGram["Words"].count(6) == 64)
assert (abGram["Words"].count(7) == 128)

#list

assert (abGram["Words"].listR(0) == [""])
assert (set(abGram["Words"].listR(1)) == set(["A","B"]))
assert (set(abGram["Words"].listR(2)) == set(["AA", "AB", "BA", "BB"]))
assert (set(abGram["Words"].listR(3)) == set(["AAA", "AAB", "ABA", "ABB",
                                              "BAA", "BAB", "BBA", "BBB"]))
test_Count_List_Until_Timeout(abGram, "Words")


the greatest test was for i = 11


In [181]:
#Grammar definition for Dyck words
dyckGram = {"Dyck" : UnionRule("Vide","Cas1"),
            "Cas1" : ProductRule("Parenthesis","Dyck", operator.add),
            "Parenthesis" : ProductRule("AtomG","CasD", operator.add),
            "CasD" : ProductRule("Dyck","AtomD", operator.add),
            "Vide" : EpsilonRule(""),
            "AtomG" : SingletonRule("("),
            "AtomD" : SingletonRule(")")
            }
init_grammar(dyckGram)

In [182]:
# tests for the Dyck words grammar

#valuation

assert (dyckGram["Dyck"].valuation() == 0)
assert (dyckGram["Cas1"].valuation() == 2)
assert (dyckGram["Vide"].valuation() == 0)
assert (dyckGram["AtomG"].valuation() == 1)
assert (dyckGram["AtomD"].valuation() == 1)
assert (dyckGram["Parenthesis"].valuation() == 2)
assert (dyckGram["CasD"].valuation() == 1)

#count

assert (dyckGram["Dyck"].count(0) == 1)
assert (dyckGram["Dyck"].count(1) == 0)
assert (dyckGram["Dyck"].count(2) == 1)
assert (dyckGram["Dyck"].count(3) == 0)
assert (dyckGram["Dyck"].count(4) == 2)
assert (dyckGram["Dyck"].count(5) == 0)
assert (dyckGram["Dyck"].count(6) == 5)
assert (dyckGram["Dyck"].count(7) == 0)
assert (dyckGram["Dyck"].count(8) == 14)
assert (dyckGram["Dyck"].count(9) == 0)

#list
assert (dyckGram["Dyck"].listR(0) == [""])
assert (set(dyckGram["Dyck"].listR(1)) == set([]))
assert (set(dyckGram["Dyck"].listR(2)) == set(["()"]))
assert (set(dyckGram["Dyck"].listR(4)) == set(["()()","(())"]))
assert (set(dyckGram["Dyck"].listR(6)) == set(["()()()","()(())","(())()",
                                             "(()())","((()))"]))
test_Count_List_Until_Timeout(dyckGram, "Dyck")

the greatest test was for i = 14


In [183]:
#Grammar definition for the (A,B) words without 3 times the same letter in a row
ab3Gram = {"Words" : UnionRule("Vide","Cas1"),
              "Cas1" : UnionRule("CasA","CasB"),
              "CasA" : ProductRule("AtomA","CasA1",operator.add),
              "CasA1" : UnionRule("CasB","CasA2"),
              "CasA2" : UnionRule("CasAB","Vide"),
              "CasB" : ProductRule("AtomB","CasB1",operator.add),
              "CasB1" : UnionRule("CasA","CasB2"),
              "CasB2" : UnionRule("CasBA","Vide"),
              "CasAB" : ProductRule("AtomA","CasAB1",operator.add),
              "CasAB1" : UnionRule("CasB","Vide"),
              "CasBA" : ProductRule("AtomB","CasBA1",operator.add),
              "CasBA1" : UnionRule("CasA","Vide"),
              "Vide" : EpsilonRule(""),
              "AtomA" : SingletonRule("A"),
              "AtomB" : SingletonRule("B")
             }

init_grammar(ab3Gram)

In [184]:
# tests for the (A,B) words without 3 times the same letter in a row grammar

#valuation
assert (ab3Gram["Words"].valuation() == 0)
assert (ab3Gram["Cas1"].valuation() == 1)
assert (ab3Gram["CasA"].valuation() == 1)
assert (ab3Gram["CasA1"].valuation() == 0)
assert (ab3Gram["CasA2"].valuation() == 0)
assert (ab3Gram["CasB"].valuation() == 1)
assert (ab3Gram["CasB1"].valuation() == 0)
assert (ab3Gram["CasB2"].valuation() == 0)
assert (ab3Gram["CasAB"].valuation() == 1)
assert (ab3Gram["CasAB1"].valuation() == 0)
assert (ab3Gram["CasBA"].valuation() == 1)
assert (ab3Gram["CasBA1"].valuation() == 0)
assert (ab3Gram["Vide"].valuation() == 0)
assert (ab3Gram["AtomA"].valuation() == 1)

#count

assert (ab3Gram["Words"].count(0) == 1)
assert (ab3Gram["Words"].count(1) == 2)
assert (ab3Gram["Words"].count(2) == 4)
assert (ab3Gram["Words"].count(3) == 6)
assert (ab3Gram["Words"].count(4) == 10)

#list
assert (ab3Gram["Words"].listR(0) == [""])
assert (set(ab3Gram["Words"].listR(1)) == set(["A","B"]))
assert (set(ab3Gram["Words"].listR(2)) == set(["AA","AB","BA","BB"]))
assert (set(ab3Gram["Words"].listR(3)) == set(["AAB","ABA","ABB","BAA","BAB","BBA"]))
assert (set(ab3Gram["Words"].listR(4)) == set(["AABA","AABB","ABAA","ABAB","ABBA",
                                              "BAAB","BABA","BABB","BBAA","BBAB"]))
test_Count_List_Until_Timeout(ab3Gram, "Words")

the greatest test was for i = 12


In [185]:
#Grammar of all the words{A,B} where Card(A) = Card(B)
GramABequal = { "Words"     : UnionRule("Vide", "Cas0"),
                "Cas0"      : ProductRule("Cas1","Words",operator.add),
                "Cas1"      : UnionRule("CasaB","CasbA"),
                "CasaB"     : ProductRule("AtomA","CasB",operator.add),
                "CasB"      : UnionRule("AtomB","CasaBB"),
                "CasaBB"    : ProductRule("AtomA","CasBB",operator.add),
                "CasBB"     : ProductRule("CasB","CasB",operator.add),
                "CasbA"     : ProductRule("AtomB","CasA",operator.add),
                "CasA"      : UnionRule("AtomA","CasbAA"),
                "CasbAA"    : ProductRule("AtomB", "CasAA",operator.add),
                "CasAA"     : ProductRule("CasA","CasA",operator.add),
                "AtomA"     : SingletonRule("A"),
                "AtomB"     : SingletonRule("B"),
                "Vide"      : EpsilonRule("")}

init_grammar(GramABequal)

In [186]:
# tests for the (a,b) words where Card(A) = Card(B) grammar

#valuation

assert (GramABequal["Words"].valuation() == 0)
assert (GramABequal["Cas0"].valuation() == 2)
assert (GramABequal["Cas1"].valuation() == 2)
assert (GramABequal["CasaB"].valuation() == 2)
assert (GramABequal["CasbA"].valuation() == 2)
assert (GramABequal["CasA"].valuation() == 1)
assert (GramABequal["CasB"].valuation() == 1)
assert (GramABequal["CasbAA"].valuation() == 3)
assert (GramABequal["CasAA"].valuation() == 2)
assert (GramABequal["CasaBB"].valuation() == 3)
assert (GramABequal["CasBB"].valuation() == 2)
assert (GramABequal["AtomA"].valuation() == 1)
assert (GramABequal["AtomB"].valuation() == 1)
assert (GramABequal["Vide"].valuation() == 0)

#count 

assert (GramABequal["Words"].count(0) == 1)
assert (GramABequal["Words"].count(1) == 0)
assert (GramABequal["Words"].count(2) == 2)
assert (GramABequal["Words"].count(3) == 0)
assert (GramABequal["Words"].count(4) == 6)
assert (GramABequal["Words"].count(5) == 0)
assert (GramABequal["Words"].count(6) == 20)

#list

assert (GramABequal["Words"].listR(0) == [""])
assert (set(GramABequal["Words"].listR(1)) == set([]))
assert (set(GramABequal["Words"].listR(2)) == set(["AB","BA"]))
assert (set(GramABequal["Words"].listR(4)) == set(["AABB","ABAB","BBAA",
                                                     "BABA","ABBA","BAAB"]))
test_Count_List_Until_Timeout(GramABequal, "Words")


the greatest test was for i = 14


In [191]:
#Grammar of Palyndromes{A,B,C}
PalindromeABC = {   "Words"     : UnionRule("Vide", "Mot"),
                    "Mot"       : UnionRule("ABC","ABCmotABC"),
                    "ABCmotABC" : UnionRule("AmotA", "BCmotBC"),
                    "BCmotBC"   : UnionRule("BmotB","CmotC"),
                    "AmotA"     : ProductRule("AtomA","MotA",operator.add),
                    "MotA"      : ProductRule("Words","AtomA",operator.add),
                    "BmotB"     : ProductRule("AtomB","MotB",operator.add),
                    "MotB"      : ProductRule("Words","AtomB",operator.add),
                    "CmotC"     : ProductRule("AtomC","MotC",operator.add),
                    "MotC"      : ProductRule("Words","AtomC",operator.add),
                    "ABC"       : UnionRule("AtomA","BC"),
                    "BC"        : UnionRule("AtomB","AtomC"),
                    "AtomA"     : SingletonRule("A"),
                    "AtomB"     : SingletonRule("B"),
                    "AtomC"     : SingletonRule("C"),
                    "Vide"      : EpsilonRule("")}

init_grammar(PalindromeABC)

In [195]:
# tests for the Palyndromes{A,B,C} grammar

#valuation

assert (PalindromeABC["Words"].valuation() == 0)
assert (PalindromeABC["Mot"].valuation() == 1)
assert (PalindromeABC["ABCmotABC"].valuation() == 2)
assert (PalindromeABC["BCmotBC"].valuation() == 2)
assert (PalindromeABC["AmotA"].valuation() == 2)
assert (PalindromeABC["BmotB"].valuation() == 2)
assert (PalindromeABC["CmotC"].valuation() == 2)
assert (PalindromeABC["MotA"].valuation() == 1)
assert (PalindromeABC["MotB"].valuation() == 1)
assert (PalindromeABC["MotC"].valuation() == 1)
assert (PalindromeABC["ABC"].valuation() == 1)
assert (PalindromeABC["BC"].valuation() == 1)
assert (PalindromeABC["AtomA"].valuation() == 1)
assert (PalindromeABC["AtomB"].valuation() == 1)
assert (PalindromeABC["AtomC"].valuation() == 1)
assert (PalindromeABC["Vide"].valuation() == 0)

#count n+1 = (n-1)³

assert (PalindromeABC["Words"].count(0) == 1)
assert (PalindromeABC["Words"].count(1) == 3)
assert (PalindromeABC["Words"].count(2) == 3)
assert (PalindromeABC["Words"].count(3) == 9)
assert (PalindromeABC["Words"].count(4) == 9)
assert (PalindromeABC["Words"].count(5) == 27)
assert (PalindromeABC["Words"].count(6) == 27)

#list

assert (PalindromeABC["Words"].listR(0) == [""])
assert (set(PalindromeABC["Words"].listR(1)) == set(["A","B","C"]))
assert (set(PalindromeABC["Words"].listR(2)) == set(["AA","BB","CC"]))
assert (set(PalindromeABC["Words"].listR(3)) == set(["AAA","ABA","ACA",
                                                    "BAB","BBB","BCB",
                                                    "CAC","CBC","CCC"]))
assert (set(PalindromeABC["Words"].listR(4)) == set(["AAAA","ABBA","ACCA",
                                                    "BAAB","BBBB","BCCB",
                                                    "CAAC","CBBC","CCCC"]))
test_Count_List_Until_Timeout(PalindromeABC, "Words")

the greatest test was for i = 12
